**Detection, Segmentation, and 3D Pose Estimation of Surgical Tools Using Deep Convolutional Neural Networks and Algebraic Geometry**

Article link: https://www.sciencedirect.com/science/article/abs/pii/S1361841521000402

* This code is dedicated to the evaluations of the proposed ART-Net. There are several sub-networks of the complete ART-Net, which are: <br>
&emsp; &emsp; &emsp; 1. Detection sub-network for the tool presence identification.  <br>
&emsp; &emsp; &emsp; 2. Segmentation sub-network for getting the surgical instrument masks.  <br>
&emsp; &emsp; &emsp; 3. Regression sub-network-1 for edgeLine (tool boundary) extraction.  <br>
&emsp; &emsp; &emsp; 4. Regression sub-network-2 for midLine extraction. <br>
&emsp; &emsp; &emsp; 5. Regression sub-network-3 for toolTip extraction.  <br>
    




* **Input:** The output predictions of each sub-network.

* **Output:** The quantitative metrics for that sub-network.



* This code is written by 
        ** Md. Kamrul Hasan 
        ** Medical Imaging and Applications (MAIA)
        ** Erasmus Scholar [2017-2019] 
        ** Contact: kamruleeekuet@gmail.com


 ## Import packages

In [1]:
import numpy as np 
import os
import glob
import cv2
import glob
import itertools
from skimage.morphology import skeletonize
import skimage.io as io
import skimage.transform as trans
import matplotlib.pyplot as plt
from math import atan2,degrees, atan
% matplotlib inline
from mpl_toolkits.mplot3d import Axes3D
from skimage.morphology import skeletonize
from skimage.transform import hough_line, hough_line_peaks, probabilistic_hough_line
from matplotlib import cm
from shapely.geometry import LineString
from shapely.geometry import Point
from sympy import *
import math
from sympy.geometry import *
from skimage.measure import label, regionprops
from scipy.spatial import distance
from skimage.morphology import disk
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import jaccard_similarity_score
from sklearn.metrics import precision_score, accuracy_score, roc_curve, auc

## Define utility functions

In [2]:
def Angular_Displacement(height, width, L1x1, L1y1, L1x2, L1y2, L2x1, L2y1, L2x2, L2y2):
    '''
    This code is for calculating Angular Displacement between a pair of lines.
    The predicted edge and mid line features can be evaluated quantitatively using the mean angular 
    displacement (mAD) error from the center of the image frame between the true andpredicted line.
    Angular displacement is calculated by the arc length distance between true point and predicted point.
    More details explanations of this metric see the paper please. 
    
    Input Arguments: 
        height = Height of the Image
        width  = Width of the Image
        L1x1 =  Initial X co-ordinate of the first line
        L1y1 =  Initial Y co-ordinate of the first line
        L1x2 =  Final X co-ordinate of the first line
        L1y2 =  Final Y co-ordinate of the first line
        L2x1 =  Initial X co-ordinate of the second line
        L2y1 =  Initial Y co-ordinate of the second line
        L2x2 =  Final X co-ordinate of the second line
        L2y2 =  Final Y co-ordinate of the second line
        
    Output Arguments:  
    
        AngularDisplacement: The average Angular Displacement of left and right side Displacement.
        
    Author: Md. Kamrul Hasan, 
        Erasmus Scholar on Medical Imaging and Application (MAIA)
        E-mail: kamruleeekuet@gmail.com
        
    '''

    center_x = int(height/2)
    center_y = int(width/2)
    centerPoint = Point(center_x,center_y)
    radius = np.sqrt((center_x)**2+(center_y)**2)

    circle = Circle(centerPoint, radius)

    lineApprox = Line((L1x1,L1y1),(L1x2,L1y2))
    lineTrue = Line((L2x1,L2y1),(L2x2,L2y2))

    intersectionApprox= intersection(circle, lineApprox)
    intersectionTrue= intersection(circle, lineTrue)

    intXY1P = intersectionApprox[0].evalf()
    X_P_1 = intXY1P.x
    Y_P_1 = intXY1P.y
#         print(X_P_1)
#         print(Y_P_1)

    intXY2P = intersectionApprox[1].evalf()
    X_P_2 = intXY2P.x
    Y_P_2 = intXY2P.y
#         print(X_P_2)
#         print(Y_P_2)


    intXY1G = intersectionTrue[0].evalf()
    X_G_1 = intXY1G.x
    Y_G_1 = intXY1G.y
#         print(X_G_1)
#         print(Y_G_1)


    intXY2G = intersectionTrue[1].evalf()
    X_G_2 = intXY2G.x
    Y_G_2 = intXY2G.y
#         print(X_G_2)
#         print(Y_G_2)

    slope1_P, intercept=slope_intercept(X_P_1, Y_P_1, center_x, center_y)
    slope1_G, intercept=slope_intercept(X_G_1, Y_G_1, center_x, center_y)

    slope2_P, intercept=slope_intercept(X_P_2, Y_P_2, center_x, center_y)
    slope2_G, intercept=slope_intercept(X_G_2, Y_G_2, center_x, center_y)

#         print(slope1_P)
#         print(slope1_G)
#         print(slope2_P)
#         print(slope2_G)

    theta_1=atan(abs((slope1_P-slope1_G)/(1+slope1_G*slope1_P)))
    theta_2=atan(abs((slope2_P-slope2_G)/(1+slope2_G*slope2_P)))
    
    AngularDisplacement=(theta_1+theta_2)/2
    
    return AngularDisplacement



def Performance_Metrics(y_true, y_pred):
    ''' 
    The Dice similarity coefficient (DSC) is a statistical validation metric for the image
    segmentations and determines the spatial overlap accuracy of the segmentation. It also 
    same as F1-score. 
    
    The Intersection over Union (IoU) also referred to as the Jaccard index (JI), is essentially
    a method to quantify the percent overlap between the GT mask and prediction output.
    This metric is closely related to the DSC. The IoU metric measures the number of pixels
    common between the target and prediction masks divided by the total number of pixels present
    across both masks.
    
    Mathematically, they are related as follows:
    IoU = DSC/(2-DSC) or DSC = 2*IoU/(IoU+1)
    
    The Sensitivity also called the true positive rate, the recall, or probability of detection that 
    measures the proportion of actual positives that are correctly identified as such class 
    (e.g., the percentage of sick people who are correctly identified as having the condition).
    
    The Specificity also called the true negative rate measures the proportion of actual negatives that
    are correctly identified as such class. (e.g., the percentage of healthy people who are correctly
    identified as not having the condition).
    
  
    Input Arguments: 
        y_true: True Labels of the 2D images so called ground truth (GT).
        y_pred: Predicted Labels of the 2D images so called Predicted/ segmented Mask.
        
    Output Arguments: 
        dsc: The DSC between y_true and y_pred
        iou: The IoU between y_true and y_pred
        accuracy: The accuracy of pixels clasification.
        sensitivity: The accuracy of foreground pixels clasification.
        specificity: The accuracy of background pixels clasification.
        balancedAccuracy: The balanced accuracy of both foreground & background pixels clasification.
        
    Author: Md. Kamrul Hasan, 
            Erasmus Scholar on Medical Imaging and Application (MAIA)
            E-mail: kamruleeekuet@gmail.com

    '''
    if y_true.shape != y_pred.shape:
        raise ValueError("Shape mismatch!! y_true and y_pred must have the same shape.")

    y_true_f = (y_true/y_true.max()).flatten()
    y_pred_f = (y_pred/y_pred.max()).flatten()
    
    intersection = np.sum(y_true_f * y_pred_f)
    
    dsc = (2. * intersection ) / (np.sum(y_true_f) + np.sum(y_pred_f))
    iou = (intersection) / (np.sum(y_true_f) + np.sum(y_pred_f)-intersection)
    
    y_true = np.asarray(y_true).astype(np.bool)
    y_pred = np.asarray(y_pred).astype(np.bool)

        
    y_true=y_true.flatten()
    y_pred=y_pred.flatten()
    
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    accuracy = (tn+tp)/(tn+fp+fn+tp)
    sensitivity = (tp)/(fn+tp)
    specificity = (tn)/(tn+fp)
    balancedAccuracy = (sensitivity+specificity)/2

    
    return dsc, iou, accuracy, sensitivity, specificity, balancedAccuracy



def slope_intercept(xi, yi, xf, yf):
    '''
    This function is dedicated for calculating the slope of the line. 
    
    Input Arguments: 
         xi: The initial X co-ordinate
         yi: The initial Y co-ordinate
         xf: The final X co-ordinate
         yf: The final Y co-ordinate
         
    Output Arguments:
        slope: Slope of the st. line connecting those two points
        intercept: The intersecting length from Y-axis
         
    Author: Md. Kamrul Hasan, 
        Erasmus Scholar on Medical Imaging and Application (MAIA)
        E-mail: kamruleeekuet@gmail.com
        
    '''
    
    slope = (yf - yi) / (xf - xi)
    intercept = yi - slope * xi 
    
    return slope,intercept

## Evaluations of Segmentation Sub-network

In [3]:
CurrentDirectory=os.getcwd()
PredictionPath = CurrentDirectory+'/Mask_V1/'
Prediction = glob.glob(PredictionPath+"*.png")
Prediction.sort()
# print(len(Prediction))

OriginalmaskGT = glob.glob(CurrentDirectory+'/Test_Tool_GT/'+"*.png")
OriginalmaskGT.sort()
# print(len(OriginalmaskGT))

OriginalImage = glob.glob(CurrentDirectory+'/TestImage/'+"*.png")
OriginalImage.sort()
# print(len(OriginalImage))

masksavePath = CurrentDirectory+'/Segmentation/'

i=0

dice=[]
sensitivity=[]
specificity=[]
accuracy = []
BalancedAccuracy=[]
IoU=[]

for Pred in Prediction:
    
    filename, file_extension = os.path.splitext(Pred) 
    
    if filename[-4:]=='tool':
        #----------------------------------Read the Prediction from SIMO------------------------------
        imgPred = cv2.imread(Pred,0)
        
        # Apply the threshold since output was sigmoid activations of segmentation sub-network
        threshold = 30
        imgPred[imgPred<threshold] = 0
        imgPred[imgPred>=threshold] = 255
        
        #--------------------------------Read Original Image with Mask GT----------------------------
        orimg = cv2.imread(OriginalImage[i],-1)
        orimg = cv2.resize(orimg,(256,192))
        
        truemask = cv2.imread(OriginalmaskGT[i],0)
        truemask = cv2.resize(truemask,(256,192))
        i=i+1
        
        #-------------------------------Overlay with original Image------------------------------------       
        temp = cv2.merge((imgPred, imgPred, imgPred))
        temp [:,:,0]=0
        temp [:,:,2]=0       

        overlay = cv2.addWeighted(temp, 0.5, orimg, 0.8, 0)
        
     
        # Call the Performance Evaluation functions by passing predicted and true mask
        dsc, iou, acc, sen, spe, bAcc = Performance_Metrics(imgPred, truemask)

        dice.append(dsc)
        IoU.append(iou)
        accuracy.append(acc)
        sensitivity.append(sen)
        specificity.append(spe)
        BalancedAccuracy.append(bAcc)
        
        font                   = cv2.FONT_HERSHEY_SIMPLEX
        bottomLeftCornerOfText = (80,185)
        fontScale              = 1
        fontColor              = (255,255,255)
        lineType               = 2
        

        cv2.putText(overlay, str(round(dsc,4)), 
            bottomLeftCornerOfText, 
            font, 
            fontScale,
            fontColor,
            lineType)
        
        savelinePath=masksavePath+'ImgID_'+filename[-8:]+'_Overlay_with_OrgImg.png'
        cv2.imwrite(savelinePath, overlay)  
        print('ImageID: '+ filename[-8:-5]+'. Dice co-efficient: '+ str(dsc)
              + '  Balanced Accuracy: ' + str(bAcc))
        
print()
print('-----------------------Final Segmentation metrics are as follows:---------------------------')
print("DSC = %0.4f (+/-%0.04f)" % (np.mean(dice)*100, np.std(dice)*100))
print("Sensitivity = %0.4f (+/-%0.04f)" % (np.mean(sensitivity)*100, np.std(sensitivity)*100))
print("Specificity = %0.4f (+/-%0.04f)" % (np.mean(specificity)*100, np.std(specificity)*100))
print("Accuracy = %0.8f (+/-%0.04f)" % (np.mean(accuracy)*100, np.std(accuracy)*100))
print("IoU = %0.4f (+/-%0.04f)" % (np.mean(IoU)*100, np.std(IoU)*100))
print("Balanced Accuracy = %0.4f (+/-%0.04f)" % (np.mean(BalancedAccuracy)*100, np.std(BalancedAccuracy)*100))     

ImageID: 011. Dice co-efficient: 0.8947923197943348  Balanced Accuracy: 0.9671406817990115
ImageID: 015. Dice co-efficient: 0.9373013006956873  Balanced Accuracy: 0.9616706615020947
ImageID: 017. Dice co-efficient: 0.9683049175948321  Balanced Accuracy: 0.9878567825680333
ImageID: 020. Dice co-efficient: 0.9775013781128693  Balanced Accuracy: 0.9869846389921632
ImageID: 037. Dice co-efficient: 0.976922065171593  Balanced Accuracy: 0.980856927078793
ImageID: 046. Dice co-efficient: 0.9670989173991883  Balanced Accuracy: 0.9813949857299211
ImageID: 062. Dice co-efficient: 0.9703446158678937  Balanced Accuracy: 0.9823543498609828
ImageID: 070. Dice co-efficient: 0.9619752018290765  Balanced Accuracy: 0.9734524616523708
ImageID: 075. Dice co-efficient: 0.9480056449747392  Balanced Accuracy: 0.9839093406557546
ImageID: 079. Dice co-efficient: 0.9312343646057728  Balanced Accuracy: 0.9814958859975758
ImageID: 083. Dice co-efficient: 0.9369814074393965  Balanced Accuracy: 0.9612475411706278
I

ImageID: 442. Dice co-efficient: 0.948413343706526  Balanced Accuracy: 0.9740548912291607
ImageID: 454. Dice co-efficient: 0.9508491437769201  Balanced Accuracy: 0.9814892285455026
ImageID: 456. Dice co-efficient: 0.9512922029401001  Balanced Accuracy: 0.9855570279362702
ImageID: 471. Dice co-efficient: 0.9660831889663699  Balanced Accuracy: 0.9934190947277666
ImageID: 474. Dice co-efficient: 0.9755963925468006  Balanced Accuracy: 0.989361014465673
ImageID: 483. Dice co-efficient: 0.9231967927352013  Balanced Accuracy: 0.9804689899708154
ImageID: 490. Dice co-efficient: 0.9684719063759413  Balanced Accuracy: 0.9820706613645425
ImageID: 495. Dice co-efficient: 0.9530431375916882  Balanced Accuracy: 0.9674002110187374
ImageID: 500. Dice co-efficient: 0.9019195639550899  Balanced Accuracy: 0.9449085707184595
ImageID: 502. Dice co-efficient: 0.953389937699934  Balanced Accuracy: 0.9935629899467306
ImageID: 504. Dice co-efficient: 0.9487009512855002  Balanced Accuracy: 0.9683656423926118
Im

## Evaluation of Mid-line regression sub-network

In [4]:
CurrentDirectory=os.getcwd()
Predictionpath = CurrentDirectory+'/Mask_V1/'
Prediction = glob.glob(Predictionpath+"*.png")
Prediction.sort()
# print(len(Prediction))

OriginalMidlineGT = glob.glob(CurrentDirectory+'/Test_midLine_GT/'+"*.png")
OriginalMidlineGT.sort()
# print(len(OriginalMidlineGT))

OriginalImage = glob.glob(CurrentDirectory+'/TestImage/'+"*.png")
OriginalImage.sort()
# print(len(OriginalImage))

midlinesavePath = CurrentDirectory+'/midline/'

i=0


AD_midline=[]

for Pred in Prediction:
    
    filename, file_extension = os.path.splitext(Pred) 
    
    if filename[-7:]=='midline':
        #------------------------------------Read the Prediction from SIMO--------------------------------
        imgPred = cv2.imread(Pred,0)
        tempimgPred = imgPred.copy()
        
        #-----------------------------------Read Original Image with Midline GT---------------------------
        orimg = cv2.imread(OriginalImage[i],-1)
        orimg = cv2.resize(orimg,(256,192))
        
        trueLine = cv2.imread(OriginalMidlineGT[i],0)
        trueLine = cv2.resize(trueLine,(256,192))
        i=i+1

        #-------------------------------Truncate the intensity lower than Threshold----------------------
        Threshold = (imgPred.max())-100
        imgPred[imgPred<Threshold]=0
        
        temp = cv2.merge((tempimgPred, tempimgPred, tempimgPred))
        temp [:,:,0]=0
        temp [:,:,2]=0 

        
        savelinePath=midlinesavePath+'ImgID_'+filename[-11:]+'_Prediction_.png'
        cv2.imwrite(savelinePath,cv2.addWeighted(temp,0.99,orimg,0.8,0))
                
        
        #-----------------------------Classic straight-line Hough transform----------------------------
        h, theta, d = hough_line(imgPred)        
        for _, angle, dist in zip(*hough_line_peaks(h, theta, d, min_distance=9, min_angle=10, threshold=0.5*h.max(), num_peaks=1)):
            y0 = (dist - 0 * np.cos(angle)) / np.sin(angle)
            y1 = (dist - imgPred.shape[1] * np.cos(angle)) / np.sin(angle)
        x0P = 0
        y0P = int(y0)
        x1P = imgPred.shape[1]
        y1P = int(y1)
        
        #-----------------------------Approximate Line-------------------------------------------------
        ApproximateLine=np.zeros_like(imgPred, dtype=np.uint8)
    
        cv2.line(ApproximateLine,(x0P,y0P),(x1P,y1P), 255, 1)
        
#         savelinePath=midlinesavePath+filename[-11:]+'_Approx_.png'
#         cv2.imwrite(savelinePath,ApproximateLine)  
        
        #---------------------------True Line approximation----------------------------------------------
        h, theta, d = hough_line(trueLine)        
        for _, angle, dist in zip(*hough_line_peaks(h, theta, d,min_distance=9, min_angle=10, threshold=0.5*h.max(), num_peaks=1)):
            y0 = (dist - 0 * np.cos(angle)) / np.sin(angle)
            y1 = (dist - imgPred.shape[1] * np.cos(angle)) / np.sin(angle)
        x0G = 0
        y0G = int(y0)
        x1G= imgPred.shape[1]
        y1G= int(y1)
        
        trueLine=np.zeros_like(trueLine, dtype=np.uint8)
    
        cv2.line(trueLine,(x0G,y0G),(x1G,y1G), 255, 1)   
        
        
        #-----------------------Overlay with original Image------------------------------------------       
        temp = cv2.merge((ApproximateLine, ApproximateLine, ApproximateLine))
        temp [:,:,0]=0
        temp [:,:,2]=0 
        
        temp2 = cv2.merge((trueLine, trueLine, trueLine))
        temp2 [:,:,0]=0 

        
        dummy = cv2.addWeighted(temp,0.99,orimg,0.8,0)
        
        dummy = cv2.addWeighted(temp2,0.99,dummy,0.8,0)
        
        
        #----------------------mean Angular Displacement Error Measurements--------------------------

        AngularDisplacement=Angular_Displacement(height = 192,
                                                width = 256,
                                                L1x1 = x0P,
                                                L1y1 = y0P,
                                                L1x2 = x1P,
                                                L1y2 = y1P,
                                                L2x1 = x0G,
                                                L2y1 = y0G,
                                                L2x2 = x1G,
                                                L2y2 = y1G)
        
        print('ImageID: '+ filename[-11:-8]+'.  AD in radian: '+ str(AngularDisplacement)
              + '  AD in degrees: ' + str(degrees(AngularDisplacement)))
        
        
        #------------------------ApproximateLine with True Line---------------------------------------
        
        TruePlusApproximation = ApproximateLine+trueLine
        
        font                   = cv2.FONT_HERSHEY_SIMPLEX
        bottomLeftCornerOfText = (80,185)
        fontScale              = 1
        fontColor              = (255,255,255)
        lineType               = 2

        cv2.putText(dummy,str(round (degrees(AngularDisplacement),2)), 
            bottomLeftCornerOfText, 
            font, 
            fontScale,
            fontColor,
            lineType)

#         savelinePath=midlinesavePath+filename[-11:]+'_ApproxPlusTrue_.png'
#         cv2.imwrite(savelinePath,TruePlusApproximation) 
        
        savelinePath=midlinesavePath+'ImgID_'+filename[-11:]+'_Overlay_with_Approx_true_.png'
        cv2.imwrite(savelinePath,dummy)  

        AD_midline.append(AngularDisplacement)
  
print()
print('-----------------------Final mean Angular Displacement (mAD)---------------------------')
print()
print('mAD in radians: '+ str(np.mean(AD_midline)) + '  mAD in degrees: ' + str(degrees(np.mean(AD_midline))))
print('std. AD in radians: '+ str(math.sqrt(np.mean((np.array(AD_midline)-np.mean(AD_midline))**2))) 
      + '  median AD in radians: ' + str(np.median(AD_midline)))




ImageID: 011.  AD in radian: 0.0169724338182852  AD in degrees: 0.9724488258528486
ImageID: 015.  AD in radian: 0.0187424965076547  AD in degrees: 1.073865947427297
ImageID: 017.  AD in radian: 0.0189826206770451  AD in degrees: 1.0876240488924558
ImageID: 020.  AD in radian: 0.0539304676598411  AD in degrees: 3.0899881840756698
ImageID: 037.  AD in radian: 0.0179297698816286  AD in degrees: 1.027300141858095
ImageID: 046.  AD in radian: 0.0202261334223558  AD in degrees: 1.1588720809694828
ImageID: 062.  AD in radian: 0.0184651217699458  AD in degrees: 1.0579735456130295
ImageID: 070.  AD in radian: 0.0179297698816282  AD in degrees: 1.027300141858076
ImageID: 075.  AD in radian: 0.0351043121979448  AD in degrees: 2.0113289316518532
ImageID: 079.  AD in radian: 0.0506682040304566  AD in degrees: 2.903074246452908
ImageID: 083.  AD in radian: 0.0181301593709421  AD in degrees: 1.0387816138545416
ImageID: 084.  AD in radian: 0.0176648224527610  AD in degrees: 1.0121197723911426
ImageID:

ImageID: 500.  AD in radian: 0.193536951733425  AD in degrees: 11.088850514152384
ImageID: 502.  AD in radian: 0.0527121216032790  AD in degrees: 3.0201820970482585
ImageID: 504.  AD in radian: 0.0175953106877669  AD in degrees: 1.008137041630472
ImageID: 507.  AD in radian: 0.0878251937417086  AD in degrees: 5.032012936318674
ImageID: 512.  AD in radian: 0.0774178818744060  AD in degrees: 4.435717890245818
ImageID: 530.  AD in radian: 0.0351485157255794  AD in degrees: 2.013861607224903
ImageID: 531.  AD in radian: 0.0528735604390396  AD in degrees: 3.029431860986844
ImageID: 551.  AD in radian: 0.353258027475910  AD in degrees: 20.240194053486125
ImageID: 552.  AD in radian: 0.0315522536222965  AD in degrees: 1.8078109666839515
ImageID: 558.  AD in radian: 0.0268605789526515  AD in degrees: 1.5389978092648608
ImageID: 559.  AD in radian: 0.0527337365292473  AD in degrees: 3.02142054108073
ImageID: 560.  AD in radian: 0.0492504914566736  AD in degrees: 2.821845299412517
ImageID: 562. 

## Evaluation of Tip-point regression sub-network

In [5]:
PredictionPath = CurrentDirectory+'/Mask_V1/'
Prediction = glob.glob(PredictionPath+"*.png")
Prediction.sort()
# print(len(Prediction))

OriginaltipGT = glob.glob(CurrentDirectory+'/Test_tip_GT/'+"*.png")
OriginaltipGT.sort()
# print(len(OriginaltipGT))

OriginalImage = glob.glob(CurrentDirectory+'/TestImage/'+"*.png")
OriginalImage.sort()
# print(len(OriginalImage))

tipsavePath = CurrentDirectory+'/TIP/'

i=0

Euclidean_Tip=[]

for Pred in Prediction:
    
    filename, file_extension = os.path.splitext(Pred) 
    
    if filename[-3:]=='tip':
        #--------------------------Read the Prediction from SIMO----------------------------------
        imgPred = cv2.imread(Pred,0)
        temp = imgPred.copy()
        
        #-------------------------Read Original Image with TIP GT---------------------------------
        orimg = cv2.imread(OriginalImage[i],-1)
        orimg = cv2.resize(orimg,(256,192))
        
        truepoint = cv2.imread(OriginaltipGT[i],0)
        truepoint = cv2.resize(truepoint,(256,192))
        i=i+1

        #----------------------Truncate the intensity lower than Threshold-----------------------
        Threshold = (imgPred.max())-100
        imgPred[imgPred<Threshold]=0
               
#         savelinePath=tipsavePath+filename[-7:]+'_Pred_.png'
#         cv2.imwrite(savelinePath,imgPred)
        
        #-----------------------Overlay with original Image--------------------------------------       
        temp = cv2.merge((imgPred, imgPred, imgPred))
        temp [:,:,0]=0
        temp [:,:,2]=0   
        
        temp2 = cv2.merge((truepoint, truepoint, truepoint))
        temp2 [:,:,0]=0

        dummy = cv2.addWeighted(temp, 0.8, orimg, 0.8, 0)
        dummy = cv2.addWeighted(temp2, 0.8, dummy, 0.8, 0)
        
                
        #------------------------Euclidean Distance-----------------------------------------------
        XY_PredictedPoint = np.argwhere(imgPred.max() == imgPred)
        XY_truePoint = np.argwhere(truepoint.max() == truepoint)
        
        X_P = XY_PredictedPoint[0][1]
        Y_P = XY_PredictedPoint[0][0]
        
        X_G = XY_truePoint[0][1]
        Y_G = XY_truePoint[0][0]  

        euclidean_distance = distance.euclidean((X_P, Y_P), (X_G, Y_G))
        print('ImageID: '+ filename[-7:-4]+ '.   Euclidean Distance in pixels: '+ str(euclidean_distance))
        Euclidean_Tip.append(euclidean_distance)
        
        #--------------------Approximate point with True point---------------------------------------
        
        font                   = cv2.FONT_HERSHEY_SIMPLEX
        bottomLeftCornerOfText = (80,185)
        fontScale              = 1
        fontColor              = (255,255,255)
        lineType               = 2

        cv2.putText(dummy,str(round(euclidean_distance,2)), 
            bottomLeftCornerOfText, 
            font, 
            fontScale,
            fontColor,
            lineType)

        savelinePath=tipsavePath+'ImgID_'+filename[-7:]+'_Overlay_Pred_True.png'
        cv2.imwrite(savelinePath,dummy)  

        
print()
print('-----------------------Final Mean Euclidean Distance (mED)---------------------------')
print()
print('Mean Euclidean Distance (mED) in pixels: '+ str(np.mean(Euclidean_Tip)))
print('std. Euclidean Distance (ED): '+ str(math.sqrt(np.mean((np.array(Euclidean_Tip)-np.mean(Euclidean_Tip))**2))) 
      + '  median Euclidean Distance (ED): ' + str(np.median(Euclidean_Tip)))
                

ImageID: 011.   Euclidean Distance in pixels: 2.23606797749979
ImageID: 015.   Euclidean Distance in pixels: 2.8284271247461903
ImageID: 017.   Euclidean Distance in pixels: 6.082762530298219
ImageID: 020.   Euclidean Distance in pixels: 3.1622776601683795
ImageID: 037.   Euclidean Distance in pixels: 1.0
ImageID: 046.   Euclidean Distance in pixels: 1.4142135623730951
ImageID: 062.   Euclidean Distance in pixels: 0.0
ImageID: 070.   Euclidean Distance in pixels: 7.280109889280518
ImageID: 075.   Euclidean Distance in pixels: 1.0
ImageID: 079.   Euclidean Distance in pixels: 2.23606797749979
ImageID: 083.   Euclidean Distance in pixels: 5.0990195135927845
ImageID: 084.   Euclidean Distance in pixels: 5.385164807134504
ImageID: 085.   Euclidean Distance in pixels: 28.0178514522438
ImageID: 091.   Euclidean Distance in pixels: 4.0
ImageID: 108.   Euclidean Distance in pixels: 1.4142135623730951
ImageID: 110.   Euclidean Distance in pixels: 1.4142135623730951
ImageID: 114.   Euclidean Dis

ImageID: 887.   Euclidean Distance in pixels: 96.87620966986684
ImageID: 892.   Euclidean Distance in pixels: 6.4031242374328485
ImageID: 893.   Euclidean Distance in pixels: 27.018512172212592
ImageID: 898.   Euclidean Distance in pixels: 27.85677655436824
ImageID: 952.   Euclidean Distance in pixels: 1.4142135623730951
ImageID: 956.   Euclidean Distance in pixels: 2.8284271247461903
ImageID: 958.   Euclidean Distance in pixels: 1.0

-----------------------Final Mean Euclidean Distance (mED)---------------------------

Mean Euclidean Distance (mED) in pixels: 9.290264508988828
std. Euclidean Distance (ED): 18.270587346234908  median Euclidean Distance (ED): 3.1622776601683795


## Evaluation of Edge-Line regression sub-network

In [6]:
testGene = CurrentDirectory+'/Mask_V1/'
Prediction = glob.glob(testGene+"*.png")
Prediction.sort()
# print(len(Prediction))

OriginalupGT = glob.glob(CurrentDirectory+'/Test_upLine_GT/'+"*.png")
OriginalupGT.sort()
# print(len(OriginalupGT))


OriginallowGT = glob.glob(CurrentDirectory+'/Test_lowerLine_GT/'+"*.png")
OriginallowGT.sort()
# print(len(OriginallowGT))


OriginalImage = glob.glob(CurrentDirectory+'/TestImage/'+"*.png")
OriginalImage.sort()
# print(len(OriginalImage))

midlinesavePath = CurrentDirectory+'/BothLine/'

i=0

AngularDisplacement_Line_1 = []
AngularDisplacement_Line_2 = []
bothError=[]

for Pred in Prediction:
    
    filename, file_extension = os.path.splitext(Pred) 
    
    if filename[-8:]=='bothline':
        #--------------------------Read the Prediction from SIMO----------------------------------------
        imgPred = cv2.imread(Pred,0)
        temp = imgPred.copy()
        temp2 = imgPred.copy()
        
        #-------------------------Read Original Image with Midline GT-----------------------------------
        orimg = cv2.imread(OriginalImage[i],-1)
        orimg = cv2.resize(orimg,(256,192))
        
        up = cv2.imread(OriginalupGT[i],0)
        up = cv2.resize(up,(256,192))
        
        up[up<100]=0
        up[up>100]=255
        
        low = cv2.imread(OriginallowGT[i],0)
        low = cv2.resize(low,(256,192))
        low[low<100]=0
        low[low>100]=255
        
        trueline=up+low
        
        i=i+1

        #-----------------------Truncate the intensity lower than Threshold--------------------------
        imgPredtemp=imgPred.copy()
        
        Threshold = (imgPred.max())-100
        imgPred[imgPred<Threshold]=0
        imgPredtemp[imgPredtemp<Threshold-30]=0
        
        
        
#         savelinePath=midlinesavePath+filename[-13:]+'_Pred_.png'
#         cv2.imwrite(savelinePath,imgPred)
        
#         savelinePath=midlinesavePath+filename[-13:]+'_true_.png'
#         cv2.imwrite(savelinePath,trueline)
        
#         savelinePath=midlinesavePath+filename[-13:]+'_truePLUSPred_.png'
#         cv2.imwrite(savelinePath,trueline+imgPred)
                
        
        #-----------------------Classic straight-line Hough transform-------------------------------
        h, theta, d = hough_line(imgPred)        
        for _, angle, dist in zip(*hough_line_peaks(h, theta, d,min_distance=9, min_angle=10, threshold=0.5*h.max(), num_peaks=1)):
            y0 = (dist - 0 * np.cos(angle)) / np.sin(angle)
            y1 = (dist - imgPred.shape[1] * np.cos(angle)) / np.sin(angle)
        x0P1 = 0
        y0P1 = int(y0)
        x1P1 = imgPred.shape[1]
        y1P1 = int(y1)
        
        #-------------------------Approximate Line_1------------------------------------------------
        ApproximateLine1=np.zeros_like(imgPred, dtype=np.uint8)
    
        cv2.line(ApproximateLine1,(x0P1,y0P1),(x1P1,y1P1), 255, 1)       
        
        #------------------------Classic straight-line Hough transform------------------------------
        temp=np.zeros_like(imgPred, dtype=np.uint8)
        temp=cv2.line(temp,(x0P1,y0P1),(x1P1,y1P1), 255, 15)
        
        temp = 255-temp
        
        imgPred=imgPred & temp
        
        h, theta, d = hough_line(imgPred)        
        for _, angle, dist in zip(*hough_line_peaks(h, theta, d,min_distance=9, min_angle=10, threshold=0.5*h.max(), num_peaks=1)):
            y0 = (dist - 0 * np.cos(angle)) / np.sin(angle)
            y1 = (dist - imgPred.shape[1] * np.cos(angle)) / np.sin(angle)
        x0P2 = 0
        y0P2 = int(y0)
        x1P2 = imgPred.shape[1]
        y1P2 = int(y1)
        
        #-------------------------Approximate Line_2------------------------------------------------
        ApproximateLine2=np.zeros_like(imgPred, dtype=np.uint8)
    
        cv2.line(ApproximateLine2,(x0P2,y0P2),(x1P2,y1P2), 255, 1)
        
        ApproximateLine = ApproximateLine2 + ApproximateLine1
        

        cv2.imwrite(midlinesavePath+filename[-13:]+'_Line_1_.png',ApproximateLine1) 
        cv2.imwrite(midlinesavePath+filename[-13:]+'_Line_2_.png',ApproximateLine2) 
        
        
        #-----------------------True Line-1 approximation------------------------------------------
        h, theta, d = hough_line(trueline)        
        for _, angle, dist in zip(*hough_line_peaks(h, theta, d,min_distance=9, min_angle=10, threshold=0.5*h.max(), num_peaks=1)):
            y0 = (dist - 0 * np.cos(angle)) / np.sin(angle)
            y1 = (dist - imgPred.shape[1] * np.cos(angle)) / np.sin(angle)
        x0G1 = 0
        y0G1 = int(y0)
        x1G1= imgPred.shape[1]
        y1G1= int(y1)
        
        up=np.zeros_like(trueline, dtype=np.uint8)
    
        cv2.line(up,(x0G1,y0G1),(x1G1,y1G1), 255, 1)  
        
        temp=np.zeros_like(trueline, dtype=np.uint8)
    
        cv2.line(temp,(x0G1,y0G1),(x1G1,y1G1), 255, 15) 
        temp = 255-temp
        
        trueline= trueline & temp
        
        labelImage=label(trueline)
        regions=regionprops(labelImage)
        Area=[]
        for region in range(len(regions)):
            maxThreshold=regions[region].area,
            Area.append(maxThreshold)
        X= sorted( [(x,i) for (i,x) in enumerate(Area)], reverse=True )[:2]

        index=np.array(X)
        V=index.T[1].tolist()
        trueline[labelImage!=(V[0]+1)]=0
        
        h, theta, d = hough_line(trueline)        
        for _, angle, dist in zip(*hough_line_peaks(h, theta, d,min_distance=9, min_angle=10, threshold=0.5*h.max(), num_peaks=1)):
            y0 = (dist - 0 * np.cos(angle)) / np.sin(angle)
            y1 = (dist - imgPred.shape[1] * np.cos(angle)) / np.sin(angle)
        x0G2 = 0
        y0G2 = int(y0)
        x1G2= imgPred.shape[1]
        y1G2= int(y1)
        
        low=np.zeros_like(trueline, dtype=np.uint8)
    
        cv2.line(low,(x0G2,y0G2),(x1G2,y1G2), 255, 1)   
        
        trueline = up+low

#         savelinePath=midlinesavePath+filename[-13:]+'_ApproxPLUStrue_.png'
#         cv2.imwrite(savelinePath,ApproximateLine+trueline) 
        
        #-------------------Overlay with original Image-------------------------------       
        temp = cv2.merge((ApproximateLine, ApproximateLine, ApproximateLine))
        temp [:,:,0]=0
        temp [:,:,2]=0  
        
        temp1 = cv2.merge((trueline, trueline, trueline))
        temp1 [:,:,0]=0

        
        dummy1 = cv2.addWeighted(temp,0.99,orimg,0.8,0)
        dummy2 = cv2.addWeighted(temp1,0.99,dummy1,0.8,0)
        
#         savelinePath=midlinesavePath+filename[-13:]+'_overlay_.png'
#         cv2.imwrite(savelinePath,dummy2) 
        
        
        AngularDisplacement_approx1_up=Angular_Displacement(height = 192,
                                                            width = 256,
                                                            L1x1 = x0P1,
                                                            L1y1 = y0P1,
                                                            L1x2 = x1P1,
                                                            L1y2 = y1P1,
                                                            L2x1 = x0G1,
                                                            L2y1 = y0G1,
                                                            L2x2 = x1G1,
                                                            L2y2 = y1G1)
        
        AngularDisplacement_approx1_low=Angular_Displacement(height = 192,
                                                            width = 256,
                                                            L1x1 = x0P1,
                                                            L1y1 = y0P1,
                                                            L1x2 = x1P1,
                                                            L1y2 = y1P1,
                                                            L2x1 = x0G2,
                                                            L2y1 = y0G2,
                                                            L2x2 = x1G2,
                                                            L2y2 = y1G2)
        
        AngularDisplacement_approx2_up=Angular_Displacement(height = 192,
                                                            width = 256,
                                                            L1x1 = x0P2,
                                                            L1y1 = y0P2,
                                                            L1x2 = x1P2,
                                                            L1y2 = y1P2,
                                                            L2x1 = x0G1,
                                                            L2y1 = y0G1,
                                                            L2x2 = x1G1,
                                                            L2y2 = y1G1)
        
        AngularDisplacement_approx2_low=Angular_Displacement(height = 192,
                                                            width = 256,
                                                            L1x1 = x0P2,
                                                            L1y1 = y0P2,
                                                            L1x2 = x1P2,
                                                            L1y2 = y1P2,
                                                            L2x1 = x0G2,
                                                            L2y1 = y0G2,
                                                            L2x2 = x1G2,
                                                            L2y2 = y1G2)
        
        if AngularDisplacement_approx1_up>AngularDisplacement_approx1_low:
            line_1_Error = AngularDisplacement_approx1_low
        else:
            line_1_Error = AngularDisplacement_approx1_up
    
        AngularDisplacement_Line_1.append(line_1_Error)
        
        if AngularDisplacement_approx2_up>AngularDisplacement_approx2_low:
            line_2_Error = AngularDisplacement_approx2_low
        else:
            line_2_Error = AngularDisplacement_approx2_up
    
        AngularDisplacement_Line_2.append(line_2_Error)

        Error= (line_1_Error+line_2_Error)/2
        
        bothError.append(Error)
        
        font                   = cv2.FONT_HERSHEY_SIMPLEX
        bottomLeftCornerOfText = (80,185)
        fontScale              = 1
        fontColor              = (255,255,255)
        lineType               = 2

        cv2.putText(dummy2,str(round(degrees(Error),2)), 
            bottomLeftCornerOfText, 
            font, 
            fontScale,
            fontColor,
            lineType)
        savelinePath=midlinesavePath+'ImgID_'+filename[-12:]+'_Pred_True-EdgeLine.png'
        cv2.imwrite(savelinePath,dummy2) 
        
        print('Angular Displacement (AD) in radian: '+ str(Error)
              + '  Angular Displacement (AD) in degrees: ' + str(degrees(Error)))

        
  
print()
print('-----------------------Final mean Angular Displacement (mAD)---------------------------')
print()
print('mAD in radians: '+ str(np.mean(bothError)) + '  mAD in degrees: ' + str(degrees(np.mean(bothError))))
print('std. AD in radians: '+ str(math.sqrt(np.mean((np.array(bothError)-np.mean(bothError))**2))) 
      + '  median AD in radians: ' + str(np.median(bothError)))

Angular Displacement (AD) in radian: 0.0279385291137560  Angular Displacement (AD) in degrees: 1.6007598040215951
Angular Displacement (AD) in radian: 0.0168614780516564  Angular Displacement (AD) in degrees: 0.9660915287123848
Angular Displacement (AD) in radian: 0.0519033149317498  Angular Displacement (AD) in degrees: 2.9738408883276097
Angular Displacement (AD) in radian: 0.0361090711512959  Angular Displacement (AD) in degrees: 2.0688973791068497
Angular Displacement (AD) in radian: 0.0256499388713313  Angular Displacement (AD) in degrees: 1.4696332420958365
Angular Displacement (AD) in radian: 0.00886677533916934  Angular Displacement (AD) in degrees: 0.508028804825082
Angular Displacement (AD) in radian: 0.00886677533916957  Angular Displacement (AD) in degrees: 0.5080288048250956
Angular Displacement (AD) in radian: 0.0158184453595639  Angular Displacement (AD) in degrees: 0.9063301575613114
Angular Displacement (AD) in radian: 0.0259585338270951  Angular Displacement (AD) in d

Angular Displacement (AD) in radian: 0.0792745669286464  Angular Displacement (AD) in degrees: 4.54209810773881
Angular Displacement (AD) in radian: 0.00806625589068716  Angular Displacement (AD) in degrees: 0.46216241900891303
Angular Displacement (AD) in radian: 0.0206873027662838  Angular Displacement (AD) in degrees: 1.1852951380173744
Angular Displacement (AD) in radian: 0  Angular Displacement (AD) in degrees: 0.0
Angular Displacement (AD) in radian: 0.0331568682419337  Angular Displacement (AD) in degrees: 1.8997486121341576
Angular Displacement (AD) in radian: 0.0440314159081833  Angular Displacement (AD) in degrees: 2.5228142975240964
Angular Displacement (AD) in radian: 0.00885069437557734  Angular Displacement (AD) in degrees: 0.507107433480757
Angular Displacement (AD) in radian: 0.0441688588205050  Angular Displacement (AD) in degrees: 2.5306891963241185
Angular Displacement (AD) in radian: 0.635956279766822  Angular Displacement (AD) in degrees: 36.43761078547991
Angular 

Angular Displacement (AD) in radian: 0.0343496174058722  Angular Displacement (AD) in degrees: 1.96808810524559
Angular Displacement (AD) in radian: 0.00891049913077693  Angular Displacement (AD) in degrees: 0.5105339935485068
Angular Displacement (AD) in radian: 0.0526819311981768  Angular Displacement (AD) in degrees: 3.018452314254109
Angular Displacement (AD) in radian: 0.0350840566883136  Angular Displacement (AD) in degrees: 2.010168376438096
Angular Displacement (AD) in radian: 0.0532189451364156  Angular Displacement (AD) in degrees: 3.0492209464548927
Angular Displacement (AD) in radian: 0.00313343158009520  Angular Displacement (AD) in degrees: 0.1795324049324639
Angular Displacement (AD) in radian: 0.0448015887424350  Angular Displacement (AD) in degrees: 2.5669419504223483
Angular Displacement (AD) in radian: 0.0371878387333187  Angular Displacement (AD) in degrees: 2.1307062086322914
Angular Displacement (AD) in radian: 0.0460829372789981  Angular Displacement (AD) in degr

## Evaluations of Detection Sub-network

In [7]:
PostivePrediction = np.array(np.loadtxt('labelPos.csv'))
NegativePrediction = np.array(np.loadtxt('labelNeg.csv'))
PredictedLabel = np.concatenate((PostivePrediction, NegativePrediction), axis=0)

In [8]:
truePositivelabel = np.array(np.ones((NegativePrediction.shape), dtype='float64'))
trueNegativelabel = np.array(np.zeros((NegativePrediction.shape), dtype='float64'))
trueLabel = np.concatenate((truePositivelabel, trueNegativelabel), axis=0)

In [9]:
precision = precision_score(trueLabel, PredictedLabel, average='macro') 
accuracy = accuracy_score(trueLabel, PredictedLabel, normalize=True)
fpr, tpr, thresholds = roc_curve(trueLabel, PredictedLabel)
AUC = auc(fpr, tpr)

In [10]:
print()
print('--------------Final Detection sub-network performances are as follows:--------------------')
print("Precision = %0.4f" % (precision*100))
print("Accuracy = %0.4f" % (accuracy*100))
print("Area Under ROC curve = %0.4f" % (AUC))


--------------Final Detection sub-network performances are as follows:--------------------
Precision = 100.0000
Accuracy = 100.0000
Area Under ROC curve = 1.0000
